# Gibbs free energy expressions for IN625 from ternary CALPHAD DB

This notebook extracts relevant thermodynamic functions necessary for the phase-field model of solid-state transformations in additive superalloy 625, represented as a ternary (Cr-Nb-Ni) with γ, δ, μ, and Laves phases competing. The thermodynamic database was prepared by U. Kattner after

    @Article{Du2005,
        Title                    = {A thermodynamic modeling of the Cr–Nb–Ni system },
        Author                   = {Yong Du and Shuhong Liu and Y.A. Chang and Ying Yang},
        Journal                  = {Calphad},
        Year                     = {2005},
        Volume                   = {29},
        Number                   = {2},
        Pages                    = {140 - 148},
        Doi                      = {10.1016/j.calphad.2005.06.001}
    }

This database models the phases of interest as follows:
- γ as $\mathrm{(Cr, Nb, Ni)}$
- δ as $\mathrm{(\mathbf{Nb}, Ni)_1(Cr, Nb, \mathbf{Ni})_3}$
- μ as $\mathrm{Nb_6(Cr, Nb, Ni)_7}$
- Laves as $\mathrm{(\mathbf{Cr}, Nb, Ni)_2(Cr, \mathbf{Nb})_1}$

The phase field model requires Gibbs free energies as functions of system compositions $x_\mathrm{Cr}$, $x_\mathrm{Nb}$, $x_\mathrm{Ni}$. The Calphad database represents these energies as functions of sublattice compositions $y$ in each phase. To avoid solving for internal phase equilibrium at each point in the simulation, approximations have been made to allow the following one-to-one mappings between $x$ and $y$:
- γ: no changes necessary
    * $y_\mathrm{Cr}' = x_\mathrm{Cr}$
    * $y_\mathrm{Nb}' = x_\mathrm{Nb}$
    * $y_\mathrm{Ni}' = x_\mathrm{Ni}$
- δ: eliminate Nb from the second (Ni) sublattice, $\mathrm{(\mathbf{Nb}, Ni)_1(Cr, \mathbf{Ni})_3}$
    * $y_\mathrm{Nb}'  = 4x_\mathrm{Nb}$
    * $y_\mathrm{Ni}'  = 1-4x_\mathrm{Nb}$
    * $y_\mathrm{Cr}'' = \frac{4}{3}x_\mathrm{Cr}$
    * $y_\mathrm{Ni}'' = 1-\frac{4}{3}x_\mathrm{Cr}$
    * Constraints: $x_\mathrm{Nb}\leq\frac{1}{4}$ and $x_\mathrm{Cr}\leq\frac{3}{4}$
- μ: no changes necessary
    * $y_\mathrm{Nb}'  = 1$
    * $y_\mathrm{Cr}'' = x_\mathrm{Cr}$
    * $y_\mathrm{Nb}'' = \frac{13}{7}x_\mathrm{Nb}-\frac{6}{7}$
    * $y_\mathrm{Ni}'' = x_\mathrm{Ni}$
    * Constraints: $x_\mathrm{Cr}\leq\frac{7}{13}$ and $x_\mathrm{Nb}\geq\frac{6}{13}$ and $x_\mathrm{Ni}\leq\frac{7}{13}$
- Laves: eliminate Nb from the first (Cr) sublattice, $\mathrm{(\mathbf{Cr}, Ni)_2(Cr, \mathbf{Nb})_1}$
    * $y_\mathrm{Cr}' = 1-\frac{3}{2}x_\mathrm{Ni}$ with $0\leq x_\mathrm{Ni}\leq\frac{2}{3}$
    * $y_\mathrm{Ni}' = \frac{3}{2}x_\mathrm{Ni}$ with $0\leq x_\mathrm{Ni}\leq\frac{2}{3}$
    * $y_\mathrm{Cr}'' = 1-3x_\mathrm{Nb}$   with $0\leq x_\mathrm{Nb}\leq\frac{1}{3}$
    * $y_\mathrm{Nb}'' = 3x_\mathrm{Nb}$   with $0\leq x_\mathrm{Nb}\leq\frac{1}{3}$

In [12]:
%matplotlib inline
# Numerical libraries
import numpy as np
from scipy.optimize import fsolve
from sympy.utilities.lambdify import lambdify
from scipy.spatial import ConvexHull

# Runtime / parallel libraries
import time, warnings
from itertools import chain
from multiprocessing import Pool

# Thermodynamics and computer-algebra libraries
from pycalphad import Database, calculate, Model
from sympy.utilities.codegen import codegen
from sympy.parsing.sympy_parser import parse_expr
from sympy import And, Ge, Gt, Le, Lt, Or, Piecewise, true
from sympy import Abs, cse, diff, logcombine, powsimp, simplify, symbols, sympify

# Visualization libraries
import matplotlib.pylab as plt
from ipywidgets import FloatProgress
from IPython.display import display

In [13]:
# Constants
epsilon = 1e-8 # tolerance for comparing floating-point numbers to zero
dx = 0.1 # small offset to avoid valid roots at edges of defined regions
#temp = 870.0 + 273.15 # 1143 Kelvin
temp = 1150.0 + 273.15 # Kelvin

RT = 8.3144598*temp # J/mol/K
Vm = 1.0e-5 # mol/m^3
inVm = 1.0 / Vm # m^3/mol
errslope = 5.0e5 # J/(mol/mol)m curvature of parabola outside phase-specific domain
madslope = 5.0e5 # J/(mol/mol)m curvature of parabola outside [0,1]

# Let's avoid integer arithmetic in fractions.
fr13by7 = 13.0/7
fr13by3 = 13.0/3
fr13by4 = 13.0/4
fr6by7 = 6.0/7
fr6by13 = 6.0/13
fr7by13 = 7.0/13
fr3by4 = 3.0/4
fr3by2 = 3.0/2
fr4by3 = 4.0/3
fr2by3 = 2.0/3
fr1by3 = 1.0/3
fr1by2 = 1.0/2
rt3by2 = np.sqrt(3.0)/2


In [14]:
# Read CALPHAD database from disk, specify phases and elements of interest
tdb = Database('Du_Cr-Nb-Ni_simple.tdb')
phases = ['FCC_A1', 'D0A_NBNI3', 'D85_NI7NB6', 'C14_LAVES', 'C15_LAVES', 'BCC_A2']
elements = ['CR', 'NB', 'NI']

## Export Gibbs functions using SymPy

In [15]:
c_gamma = list(set([i for c in tdb.phases['FCC_A1'].constituents for i in c]))
m_gamma = Model(tdb, c_gamma, 'FCC_A1')
g_gamma = parse_expr(str(m_gamma.ast))

In [16]:
c_delta = list(set([i for c in tdb.phases['D0A_NBNI3'].constituents for i in c]))
m_delta = Model(tdb, c_delta, 'D0A_NBNI3')
g_delta = parse_expr(str(m_delta.ast))

In [17]:
c_mu = list(set([i for c in tdb.phases['D85_NI7NB6'].constituents for i in c]))
m_mu = Model(tdb, c_mu, 'D85_NI7NB6')
g_mu = parse_expr(str(m_mu.ast))

In [18]:
c_lavesHT = list(set([i for c in tdb.phases['C14_LAVES'].constituents for i in c]))
m_lavesHT = Model(tdb, c_lavesHT, 'C14_LAVES')
g_lavesHT = parse_expr(str(m_lavesHT.ast))

In [19]:
c_lavesLT = list(set([i for c in tdb.phases['C15_LAVES'].constituents for i in c]))
m_lavesLT = Model(tdb, c_lavesLT, 'C15_LAVES')
g_lavesLT = parse_expr(str(m_lavesLT.ast))

In [20]:
c_bcc = list(set([i for c in tdb.phases['BCC_A2'].constituents for i in c]))
m_bcc = Model(tdb, c_bcc, 'BCC_A2')
g_bcc = parse_expr(str(m_bcc.ast))

## Convert sublattice to phase composition (y to x)

In [ ]:
# Declare sublattice variables used in Pycalphad expressions
# Gamma
FCC_A10CR, FCC_A10NB, FCC_A10NI, FCC_A11VA = symbols('FCC_A10CR FCC_A10NB FCC_A10NI FCC_A11VA')
# Delta
D0A_NBNI30NI, D0A_NBNI30NB, D0A_NBNI31CR, D0A_NBNI31NI = symbols('D0A_NBNI30NI D0A_NBNI30NB D0A_NBNI31CR D0A_NBNI31NI')
# Mu
D85_NI7NB60NB, D85_NI7NB61CR, D85_NI7NB61NB, D85_NI7NB61NI = symbols('D85_NI7NB60NB D85_NI7NB61CR D85_NI7NB61NB D85_NI7NB61NI')
# Laves
C14_LAVES0CR, C14_LAVES0NI, C14_LAVES1CR, C14_LAVES1NB = symbols('C14_LAVES0CR C14_LAVES0NI C14_LAVES1CR C14_LAVES1NB') 
C15_LAVES0CR, C15_LAVES0NI, C15_LAVES1CR, C15_LAVES1NB = symbols('C15_LAVES0CR C15_LAVES0NI C15_LAVES1CR C15_LAVES1NB') 
# BCC
BCC_A20CR, BCC_A20NB, BCC_A20NI, BCC_A21VA = symbols('BCC_A20CR BCC_A20NB BCC_A20NI BCC_A21VA')
# Temperature
T = symbols('T')

# Declare system variables for target expressions
GAMMA_XCR, GAMMA_XNB, GAMMA_XNI = symbols('GAMMA_XCR GAMMA_XNB GAMMA_XNI')
DELTA_XCR, DELTA_XNB, DELTA_XNI = symbols('DELTA_XCR DELTA_XNB DELTA_XNI')
MU_XCR, MU_XNB, MU_XNI = symbols('MU_XCR MU_XNB MU_XNI')
LAVES_XCR, LAVES_XNB, LAVES_XNI = symbols('LAVES_XCR LAVES_XNB LAVES_XNI')
BCC_XCR, BCC_XNB, BCC_XNI = symbols('BCC_XCR BCC_XNB BCC_XNI')

In [ ]:
# Make substitutions

g_delta_raw = inVm * g_delta.subs({
                     D0A_NBNI30NB: 4.0*DELTA_XNB,
                     D0A_NBNI30NI: 1.0 - 4.0*DELTA_XNB,
                     D0A_NBNI31CR: fr4by3 * DELTA_XCR,
                     D0A_NBNI31NI: 1.0 - fr4by3 * DELTA_XCR,
                     T: temp})

g_mu_raw = inVm * g_mu.subs({
                  D85_NI7NB60NB: 1.0,
                  D85_NI7NB61CR: fr13by7*MU_XCR,
                  D85_NI7NB61NB: fr13by7*MU_XNB - fr6by7,
                  D85_NI7NB61NI: fr13by7*MU_XNI,
                  T: temp})

g_lavesHT_raw = inVm * g_lavesHT.subs({
                  C14_LAVES0CR: 1.0 - fr3by2*LAVES_XNI,
                  C14_LAVES0NI: fr3by2 * LAVES_XNI,
                  C14_LAVES1CR: 1.0 - 3.0*LAVES_XNB,
                  C14_LAVES1NB: 3.0 * LAVES_XNB,
                  T: temp})

g_lavesLT_raw = inVm * g_lavesLT.subs({
                  C15_LAVES0CR: 1.0 - fr3by2*LAVES_XNI,
                  C15_LAVES0NI: fr3by2 * LAVES_XNI,
                  C15_LAVES1CR: 1.0 - 3.0*LAVES_XNB,
                  C15_LAVES1NB: 3.0 * LAVES_XNB,
                  T: temp})

g_gamma = inVm * simplify(Piecewise((madslope*((GAMMA_XCR-fr1by3)**2 
                                    + (GAMMA_XNB-fr1by3)**2
                                    + (1.0-GAMMA_XCR-GAMMA_XNB-fr1by3)**2),
                          Lt(GAMMA_XCR, 0.0) | 
                          Gt(GAMMA_XCR, 1.0) | 
                          Lt(GAMMA_XNB, 0.0) | 
                          Gt(GAMMA_XNB, 1.0) | 
                          Lt(GAMMA_XNI, 0.0) | 
                          Gt(GAMMA_XNI, 1.0)),
                        (g_gamma.subs({FCC_A10CR: GAMMA_XCR,
                        FCC_A10NB: GAMMA_XNB,
                        FCC_A10NI: GAMMA_XNI,
                        FCC_A11VA: 1.0,
                        T: temp}), True)))

# Delta, Mu, and Laves are not valid over the entire simplex. Restrict their domains.
g_delta = inVm * simplify(Piecewise((madslope*((DELTA_XCR-fr1by3)**2 
                                             + (DELTA_XNB-fr1by3)**2 
                                             + (1.0-DELTA_XCR-DELTA_XNB-fr1by3)**2),
                          Lt(1-DELTA_XCR-DELTA_XNB, 0.0) | 
                          Gt(1-DELTA_XCR-DELTA_XNB, 1.0) | 
                          Lt(DELTA_XCR, 0.0) | 
                          Gt(DELTA_XCR, 1.0) |
                          Lt(DELTA_XNB, 0.0) | 
                          Gt(DELTA_XNB, 1.0)),
                    (g_delta.subs({
                     D0A_NBNI30NB: 4.0*DELTA_XNB,
                     D0A_NBNI30NI: 1.0 - 4.0*DELTA_XNB,
                     D0A_NBNI31CR: fr4by3 * DELTA_XCR,
                     D0A_NBNI31NI: 1.0 - fr4by3 * DELTA_XCR,
                     T: temp}), 
                     And(Le(DELTA_XCR, 0.75), Le(DELTA_XNB, 0.25))),
                    (errslope*((DELTA_XCR-0.75+dx)**2 + (DELTA_XNB-0.25+dx)**2), True))) # Delta depends only on xcr and xnb, not xni

g_mu = inVm * simplify(Piecewise((madslope*((MU_XCR-fr1by3)**2 +
                                          (MU_XNB-fr1by3)**2 + 
                                          (MU_XNI-fr1by3)**2),
                                Lt(MU_XCR, 0.0) | 
                                Gt(MU_XCR, 1.0) | 
                                Lt(MU_XNB, 0.0) | 
                                Gt(MU_XNB, 1.0) | 
                                Lt(MU_XNI, 0.0) |
                                Gt(MU_XNI, 1.0)),
                 (g_mu.subs({
                  D85_NI7NB60NB: 1.0,
                  D85_NI7NB61CR: fr13by7*MU_XCR,
                  D85_NI7NB61NB: fr13by7*MU_XNB - fr6by7,
                  D85_NI7NB61NI: fr13by7*MU_XNI,
                  T: temp}), 
                  And(Le(MU_XCR+MU_XNI, fr7by13), Ge(MU_XNB, fr6by13))),
                (errslope*((MU_XCR+MU_XNI-fr7by13-dx)**2 + (MU_XNB-fr6by13-dx)**2), True))) # Mu depends only on xcr and xni, not xnb

g_lavesHT = inVm * simplify(Piecewise((madslope*((1-LAVES_XNB-LAVES_XNI-fr1by3)**2 + 
                                               (LAVES_XNB-fr1by3)**2 + 
                                               (LAVES_XNI-fr1by3)**2),
                                     Lt(LAVES_XNB, 0.0) | 
                                     Gt(LAVES_XNB, 1.0) | 
                                     Lt(LAVES_XNI, 0.0) | 
                                     Gt(LAVES_XNI, 1.0) | 
                                     Lt(1-LAVES_XNB-LAVES_XNI, 0.0) | 
                                     Gt(1-LAVES_XNB-LAVES_XNI, 1.0)),
                 (g_lavesHT.subs({
                  C14_LAVES0CR: 1.0 - fr3by2*LAVES_XNI,
                  C14_LAVES0NI: fr3by2 * LAVES_XNI,
                  C14_LAVES1CR: 1.0 - 3.0*LAVES_XNB,
                  C14_LAVES1NB: 3.0 * LAVES_XNB,
                  T: temp}), 
                  And(Lt(LAVES_XNB, fr1by3), Lt(LAVES_XNI, fr2by3))),
                (errslope*((LAVES_XNB-fr1by3-dx)**2 + (LAVES_XNI-fr2by3+dx)**2), True))) # Laves depends only on xnb and xni, not xcr

g_lavesLT = inVm * Piecewise((madslope*((1-LAVES_XNB-LAVES_XNI-fr1by3)**2 + 
                                      (LAVES_XNB-fr1by3)**2 + 
                                      (LAVES_XNI-fr1by3)**2),
                                     Lt(LAVES_XNB, 0.0) | 
                                     Gt(LAVES_XNB, 1.0) | 
                                     Lt(LAVES_XNI, 0.0) | 
                                     Gt(LAVES_XNI, 1.0) | 
                                     Lt(1-LAVES_XNB-LAVES_XNI, 0.0) | 
                                     Gt(1-LAVES_XNB-LAVES_XNI, 1.0)),
                 (g_lavesLT.subs({
                  C15_LAVES0CR: 1.0 - fr3by2*LAVES_XNI,
                  C15_LAVES0NI: fr3by2 * LAVES_XNI,
                  C15_LAVES1CR: 1.0 - 3.0*LAVES_XNB,
                  C15_LAVES1NB: 3.0 * LAVES_XNB,
                  T: temp}), \
                  And(Lt(LAVES_XNB, fr1by3), Lt(LAVES_XNI, fr2by3))),
                (errslope*((LAVES_XNB-fr1by3-dx)**2 + (LAVES_XNI-fr2by3+dx)**2), True))

g_bcc = inVm * g_bcc.subs({BCC_A20CR: BCC_XCR,
                    BCC_A20NB: BCC_XNB,
                    BCC_A20NI: BCC_XNI,
                    BCC_A21VA: 1.0,
                    T: temp})

In [ ]:
# Generate numerically efficient system-composition expressions
Gg = lambdify((GAMMA_XCR, GAMMA_XNB, GAMMA_XNI), g_gamma)
Gd = lambdify((DELTA_XCR, DELTA_XNB), g_delta)
Gu = lambdify((MU_XCR, MU_XNB, MU_XNI), g_mu)
Gh = lambdify((LAVES_XNB, LAVES_XNI), g_lavesHT)
Gl = lambdify((LAVES_XNB, LAVES_XNI), g_lavesLT)
Gb = lambdify((BCC_XCR, BCC_XNB, BCC_XNI), g_bcc)

Gdr = lambdify((DELTA_XCR, DELTA_XNB), g_delta_raw)
Gur = lambdify((MU_XCR, MU_XNB, MU_XNI), g_mu_raw)
Ghr = lambdify((LAVES_XNB, LAVES_XNI), g_lavesHT_raw)
Glr = lambdify((LAVES_XNB, LAVES_XNI), g_lavesLT_raw)

In [ ]:
labels = [r'$\gamma$', r'$\delta$', r'$\mu$', 'LavesHT', 'LavesLT', 'BCC', r'$\delta$', r'$\mu$', 'LavesHT', 'LavesLT']
colors = ['red', 'green', 'blue', 'cyan', 'magenta', 'yellow', 'green', 'blue', 'cyan', 'magenta']

# plot Gibbs energy curves for fixed x_Cr = 30% to check mu stability
if True:
    xcr = 0.30
    X = []
    G = [[], [], [], [], [], [], [], [], [], []]

    for xnb in np.linspace(-0.01+epsilon,1.01,501):
        xni = 1-xcr-xnb
        X.append(xnb)
        G[0].append(g_gamma.subs({GAMMA_XCR: xcr, GAMMA_XNB: xnb, GAMMA_XNI: xni}))
        G[1].append(g_delta.subs({DELTA_XCR: xcr, DELTA_XNB: xnb}))
        G[2].append(g_mu.subs({MU_XCR: xcr, MU_XNB: xnb, MU_XNI: xni}))
        G[3].append(g_lavesHT.subs({LAVES_XNB: xnb, LAVES_XNI: xni}))

    plt.figure(figsize=(20, 20))
    plt.xlabel(r'$x_\mathrm{Nb}$', fontsize=24)
    plt.ylabel(r'$\mathcal{G}$', fontsize=24)
    for i in range(4):
        plt.plot(X, G[i], color=colors[i], label=labels[i])
    plt.legend(loc='best')

In [ ]:
# plot Gibbs energy curves for fixed x_Nb = 2% to check delta stability
if True:
    xnb = 0.02
    X = []
    G = [[], [], [], [], [], [], [], [], [], []]

    for xcr in np.linspace(-0.01+epsilon,1.01,501):
        xni = 1-xcr-xnb
        X.append(xcr)
        G[0].append(g_gamma.subs({GAMMA_XCR: xcr, GAMMA_XNB: xnb, GAMMA_XNI: xni}))
        G[1].append(g_delta.subs({DELTA_XCR: xcr, DELTA_XNB: xnb}))
        G[2].append(g_mu.subs({MU_XCR: xcr, MU_XNB: xnb, MU_XNI: xni}))
        G[3].append(g_lavesHT.subs({LAVES_XNB: xnb, LAVES_XNI: xni}))

    plt.figure(figsize=(20, 20))
    plt.xlabel(r'$x_\mathrm{Cr}$', fontsize=24)
    plt.ylabel(r'$\mathcal{G}$', fontsize=24)
    for i in range(4):
        plt.plot(X, G[i], color=colors[i], label=labels[i])
    plt.legend(loc='best')

# Generate a phase diagram
Using scipy.spatial.ConvexHull, an interface to qhull. This method cannot provide phase fractions, chemical potentials, etc., but will quickly produce the correct diagram from the given Gibbs energies.

In [ ]:
labels = [r'$\gamma$', r'$\delta$', r'$\mu$', 'LavesHT', 'LavesLT', 'BCC']
colors = ['red', 'green', 'blue', 'cyan', 'magenta', 'yellow']

# triangle bounding the Gibbs simplex
XS = [0.0, 1.0, 0.5, 0.0]
YS = [0.0, 0.0,rt3by2, 0.0]
# triangle bounding three-phase coexistence
XT = [0.25, 0.4875+0.025/2,0.5375+0.4625/2, 0.25]
YT = [0.0,  0.025*rt3by2, 0.4625*rt3by2, 0.0]
# Tick marks along simplex edges
Xtick = []
Ytick = []
for i in range(20):
    # Cr-Ni edge
    xcr = 0.05*i
    xni = 1.0 - xcr
    Xtick.append(xcr/2 - 0.002)
    Ytick.append(rt3by2*xcr)
    # Cr-Nb edge
    xcr = 0.05*i
    xnb = 1.0 - xcr
    Xtick.append(xnb + xcr/2 + 0.002)
    Ytick.append(rt3by2*xcr)

In [ ]:
def computeKernelExclusive(n):
    a = n / density # index along x-axis
    b = n % density # index along y-axis

    xnb = epsilon + 1.0*a / (density-1)
    xcr = epsilon + 1.0*b / (density-1)
    xni = 1.0 - xcr - xnb

    result = [0]*9
    
    if xni>0:
        result[0] = xcr
        result[1] = xnb
        result[2] = xni
        result[3] = Gg(xcr,xnb,xni)
        result[4] = Gd(xcr,xnb)
        result[5] = Gu(xcr,xnb,xni)
        result[6] = Gh(xnb,xni)
        result[7] = Gl(xnb,xni)
        result[8] = Gb(xcr,xnb,xni)
    
    return result

In [ ]:
density = 501
allCr = []
allNb = []
allG = []
allID = []
points = []
phases = []

if __name__ == '__main__':
    starttime = time.time() # not exact, but multiprocessing makes time.clock() read from different cores

    bar = FloatProgress(min=0,max=density**2)
    display(bar)

    pool = Pool(12)

    i = 0
    for result in pool.imap(computeKernelExclusive, range(density**2)):
        xcr, xnb, xni, fg, fd, fu, fh, fl, fb = result
        f = (fg, fd, fu, fh, fl, fb)

        # Accumulate (x, y, G) points for each phase
        if (fd**2 + fu**2 + fh**2 + fl**2) > epsilon:
            for n in range(len(f)):
                allCr.append(rt3by2*xcr)
                allNb.append(xnb+xcr/2)
                allG.append(f[n])
                allID.append(n)
        i += 1
        bar.value = i

    pool.close()
    pool.join()
    
    points = np.array([allCr, allNb, allG]).T
    
    hull = ConvexHull(points)
    
    runtime = time.time() - starttime
    print "%ih:%im:%is elapsed" % (int(runtime/3600), int(runtime/60)%60, int(runtime)%60)

In [ ]:
# Prepare arrays for plotting
X = [[],[],[],[], [], []]
Y = [[],[],[],[], [], []]
tielines = []

for simplex in hull.simplices:
    for i in simplex:
        X[allID[i]].append(allNb[i])
        Y[allID[i]].append(allCr[i])
        for j in simplex:
            if allID[i] != allID[j]:
                tielines.append([[allNb[i], allNb[j]],
                                 [allCr[i], allCr[j]]])

In [ ]:
# Plot phase diagram
pltsize = 20
plt.figure(figsize=(pltsize, rt3by2*pltsize))
plt.title("Cr-Nb-Ni at %.0fK"%temp, fontsize=18)
plt.xlim([0,1])
plt.ylim([0,rt3by2])
plt.xlabel(r'$x_\mathrm{Nb}$', fontsize=24)
plt.ylabel(r'$x_\mathrm{Cr}$', fontsize=24)
plt.plot(XS, YS, '-k')
n = 0
for tie in tielines:
    plt.plot(tie[0], tie[1], '-k', alpha=0.025)
for i in range(len(labels)):
    plt.scatter(X[i], Y[i], color=colors[i], s=2.5, label=labels[i])
plt.xticks(np.linspace(0, 1, 21))
plt.scatter(Xtick, Ytick, color='black', s=3)
plt.legend(loc='best')
plt.show()

## Export C code
If the phase diagram above looks "right," proceed to code generation. 

In [ ]:
# Generate first derivatives
dGgam_dxCr = diff(g_gamma, GAMMA_XCR)
dGgam_dxNb = diff(g_gamma, GAMMA_XNB)
dGgam_dxNi = diff(g_gamma, GAMMA_XNI)

dGdel_dxCr = diff(g_delta, DELTA_XCR)
dGdel_dxNb = diff(g_delta, DELTA_XNB)

dGmu_dxCr = diff(g_mu, MU_XCR)
dGmu_dxNi = diff(g_mu, MU_XNI)

dGlavH_dxNb = diff(g_lavesHT, LAVES_XNB)
dGlavH_dxNi = diff(g_lavesHT, LAVES_XNI)
dGlavL_dxNb = diff(g_lavesLT, LAVES_XNB)
dGlavL_dxNi = diff(g_lavesLT, LAVES_XNI)

# Generate optimized second derivatives
d2Ggam_dxCrCr = powsimp(logcombine(diff(dGgam_dxCr, GAMMA_XCR)))
d2Ggam_dxCrNb = powsimp(logcombine(diff(dGgam_dxCr, GAMMA_XNB)))
d2Ggam_dxNbCr = powsimp(logcombine(diff(dGgam_dxNb, GAMMA_XCR)))
d2Ggam_dxNbNb = powsimp(logcombine(diff(dGgam_dxNb, GAMMA_XNB)))
d2Ggam_dxNiCr = powsimp(logcombine(diff(dGgam_dxNi, GAMMA_XCR)))
d2Ggam_dxNiNb = powsimp(logcombine(diff(dGgam_dxNi, GAMMA_XNB)))

d2Gdel_dxCrCr = powsimp(logcombine(diff(dGdel_dxCr, DELTA_XCR)))
d2Gdel_dxCrNb = powsimp(logcombine(diff(dGdel_dxCr, DELTA_XNB)))
d2Gdel_dxNbCr = powsimp(logcombine(diff(dGdel_dxNb, DELTA_XCR)))
d2Gdel_dxNbNb = powsimp(logcombine(diff(dGdel_dxNb, DELTA_XNB)))

d2Gmu_dxCrCr = powsimp(logcombine(diff(dGmu_dxCr, MU_XCR)))
d2Gmu_dxCrNi = powsimp(logcombine(diff(dGmu_dxCr, MU_XNI)))
d2Gmu_dxNiCr = powsimp(logcombine(diff(dGmu_dxNi, MU_XCR)))
d2Gmu_dxNiNi = powsimp(logcombine(diff(dGmu_dxNi, MU_XNI)))

d2GlavH_dxNbNb = powsimp(logcombine(diff(dGlavH_dxNb, LAVES_XNB)))
d2GlavH_dxNbNi = powsimp(logcombine(diff(dGlavH_dxNb, LAVES_XNI)))
d2GlavH_dxNiNb = powsimp(logcombine(diff(dGlavH_dxNi, LAVES_XNB)))
d2GlavH_dxNiNi = powsimp(logcombine(diff(dGlavH_dxNi, LAVES_XNI)))

d2GlavL_dxNbNb = diff(dGlavL_dxNb, LAVES_XNB)
d2GlavL_dxNbNi = diff(dGlavL_dxNb, LAVES_XNI)
d2GlavL_dxNiNb = diff(dGlavL_dxNi, LAVES_XNB)
d2GlavL_dxNiNi = diff(dGlavL_dxNi, LAVES_XNI)

# Optimize primary expressions
g_gamma   = powsimp(logcombine(g_gamma))
g_delta   = powsimp(logcombine(g_delta))
g_mu      = powsimp(logcombine(g_mu))
g_lavesHT = powsimp(logcombine(g_lavesHT))

# Optimize first derivatives
dGgam_dxCr = powsimp(logcombine(dGgam_dxCr))
dGgam_dxNb = powsimp(logcombine(dGgam_dxNb))
dGgam_dxNi = powsimp(logcombine(dGgam_dxNi))
dGdel_dxCr = powsimp(logcombine(dGdel_dxCr))
dGdel_dxNb = powsimp(logcombine(dGdel_dxNb))
dGmu_dxCr = powsimp(logcombine(dGmu_dxCr))
dGmu_dxNi = powsimp(logcombine(dGmu_dxNi))
dGlavH_dxNb = powsimp(logcombine(dGlavH_dxNb))
dGlavH_dxNi = powsimp(dGlavH_dxNi)

In [ ]:
# Write Gibbs energy functions to disk, for direct use in phase-field code
codegen([# Gibbs energies
         ('g_gam',g_gamma), ('g_mu',g_mu), ('g_lav',g_lavesHT), ('g_lavLT',g_lavesLT), ('g_del',g_delta),
         # First derivatives
         ('dg_gam_dxCr',dGgam_dxCr), ('dg_gam_dxNb',dGgam_dxNb), ('dg_gam_dxNi',dGgam_dxNi),
         ('dg_del_dxCr',dGdel_dxCr), ('dg_del_dxNb',dGdel_dxNb),
         ('dg_mu_dxCr',dGmu_dxCr), ('dg_mu_dxNi',dGmu_dxNi),
         ('dg_lav_dxNb',dGlavH_dxNb), ('dg_lav_dxNi',dGlavH_dxNi),
         ('dg_lavLT_dxNb',dGlavL_dxNb), ('dg_lavLT_dxNi',dGlavL_dxNi),
         # Second derivatives
         ('d2g_gam_dxCrCr',  d2Ggam_dxCrCr), ('d2g_gam_dxCrNb',d2Ggam_dxCrNb),
         ('d2g_gam_dxNbCr',  d2Ggam_dxNbCr), ('d2g_gam_dxNbNb',d2Ggam_dxNbNb),
         ('d2g_gam_dxNiCr',  d2Ggam_dxNiCr), ('d2g_gam_dxNiNb',d2Ggam_dxNiNb),
         ('d2g_del_dxCrCr',  d2Gdel_dxCrCr), ('d2g_del_dxCrNb',d2Gdel_dxCrNb),
         ('d2g_del_dxNbCr',  d2Gdel_dxNbCr), ('d2g_del_dxNbNb',d2Gdel_dxNbNb),
         ('d2g_mu_dxCrCr',   d2Gmu_dxCrCr),  ('d2g_mu_dxCrNi', d2Gmu_dxCrNi),
         ('d2g_mu_dxNiCr',   d2Gmu_dxNiCr),  ('d2g_mu_dxNiNi', d2Gmu_dxNiNi),
         ('d2g_lav_dxNbNb',  d2GlavH_dxNbNb),('d2g_lav_dxNbNi',d2GlavH_dxNbNi),
         ('d2g_lav_dxNiNb',  d2GlavH_dxNiNb),('d2g_lav_dxNiNi',d2GlavH_dxNiNi),
         ('d2g_lavLT_dxNbNb',d2GlavL_dxNbNb),('d2g_lavLT_dxNbNi',d2GlavL_dxNbNi),
         ('d2g_lavLT_dxNiNb',d2GlavL_dxNiNb),('d2g_lavLT_dxNiNi',d2GlavL_dxNiNi)],
        language='C', prefix='energy625hi', project='ALLOY625', to_files=True)

## Interfacial Energy
This is a naïve attempt to determine an appropriate interfacial energy for $\mu$-phase by finding the balancing point of metastable equilibrium between $\gamma$ and $\mu$.

In [ ]:
sigma_del = 1.01 # J/m^2
sigma_mu = sigma_del #/ 10 # J/m^2
sigma_lav = sigma_del #/ 10 # J/m^2
sigma_gprime = sigma_del / 20 # J / m^2: absolute minimum interfacial energy!
#if sigma_gprime > sigma_mu or sigma_gprime > sigma_lav:
#    # complain bitterly and at length about breaking physics    

rmax = 1.0e-8  # m
rmin = 1.0e-16 # m
Vm = 1.0e-5 # m**3/mol

radius = 7.0e-9

In [ ]:
def computeKernelInterfacial(n):
    a = n / density # index along x-axis
    b = n % density # index along y-axis

    xnb = epsilon + 1.0*a / (density-1)
    xcr = epsilon + 1.0*b / (density-1)
    xni = 1.0 - xcr - xnb

    result = [0]*8
    
    if xni>0:
        result[0] = xcr
        result[1] = xnb
        result[2] = xni
        result[3] = Gg(xcr,xnb,xni)
        result[4] = Gd(xcr,xnb)     + 2.0 * Vm * sigma_del / radius
        result[5] = Gu(xcr,xnb,xni) + 2.0 * Vm * sigma_mu  / radius
        result[6] = Gh(xnb,xni)     + 2.0 * Vm * sigma_lav / radius
        result[7] = Gl(xnb,xni)     + 2.0 * Vm * sigma_lav / radius

    return result

In [ ]:
density = 201
allCr = []
allNb = []
allG = []
allID = []
points = []
phases = []

if __name__ == '__main__':
    starttime = time.time() # not exact, but multiprocessing makes time.clock() read from different cores

    bar = FloatProgress(min=0, max=density**2)
    display(bar)

    pool = Pool(12)

    i = 0
    for result in pool.imap(computeKernelInterfacial, range(density**2)):
        # Excluding BCC
        xcr, xnb, xni, fg, fd, fu, fh, fl = result
        f = (fg, fd, fu, fh, fl)

        # Accumulate (x, y, G) points for each phase
        if (fd**2 + fu**2 + fh**2 + fl**2) > epsilon:
            for n in range(len(f)):
                allCr.append(rt3by2*xcr)
                allNb.append(xnb+xcr/2)
                allG.append(f[n])
                allID.append(n)
        i += 1
        bar.value = i

    pool.close()
    pool.join()
    
    points = np.array([allCr, allNb, allG]).T
    
    hull = ConvexHull(points)
    
    runtime = time.time() - starttime
    print "%ih:%im:%is elapsed" % (int(runtime/3600), int(runtime/60)%60, int(runtime)%60)

In [ ]:
# Prepare arrays for plotting
X = [[],[],[],[], [], []]
Y = [[],[],[],[], [], []]
tielines = []

for simplex in hull.simplices:
    for i in simplex:
        X[allID[i]].append(allNb[i])
        Y[allID[i]].append(allCr[i])
        for j in simplex:
            if allID[i] != allID[j]:
                tielines.append([[allNb[i], allNb[j]],
                                 [allCr[i], allCr[j]]])

In [ ]:
# Plot phase diagram
pltsize = 20
plt.figure(figsize=(pltsize, rt3by2*pltsize))
plt.title("Cr-Nb-Ni at %.0fK with r=%.0e m"%(temp, radius), fontsize=18)
plt.xlim([0,1])
plt.ylim([0,rt3by2])
plt.xlabel(r'$x_\mathrm{Nb}$', fontsize=24)
plt.ylabel(r'$x_\mathrm{Cr}$', fontsize=24)
plt.plot(XS, YS, '-k')
for tie in tielines:
    plt.plot(tie[0], tie[1], '-k', alpha=0.025)
for i in range(len(labels)):
    plt.scatter(X[i], Y[i], color=colors[i], s=2.5, label=labels[i])
plt.xticks(np.linspace(0, 1, 21))
plt.scatter(Xtick, Ytick, color='black', s=3)
plt.legend(loc='best')
plt.show()

## Scan over precipitate sizes
For the initial condition, need to know how big (or small) to make each precipitate.

In [ ]:
density = 101
lattice = 4
r0 = 2.5e-9
sigma = 1.01 # J/m^2
sA = sigma
sB = sigma
sC = sigma

def computeKernelLattice(n):
    a = n / density # index along x-axis
    b = n % density # index along y-axis

    xnb = epsilon + 1.0*a / (density-1)
    xcr = epsilon + 1.0*b / (density-1)
    xni = 1.0 - xcr - xnb

    result = [0]*8
    
    if xni>0:
        result[0] = xcr
        result[1] = xnb
        result[2] = xni
        result[3] = Gg(xcr,xnb,xni)
        result[4] = Gd(xcr,xnb)     + 2.0 * Vm * sA / rA
        result[5] = Gu(xcr,xnb,xni) + 2.0 * Vm * sB / rB
        result[6] = Gh(xnb,xni)     + 2.0 * Vm * sC / rC
        result[7] = Gl(xnb,xni)     + 2.0 * Vm * sC / rC

    return result

fig, axs = plt.subplots(lattice, lattice, sharex=True, sharey=True, figsize=(pltsize, pltsize))
fig.suptitle(r"Cr-Nb-Ni at %.0fK with $\sigma_\delta=%.2f, \sigma_\mu=%.2f, \sigma_L=%.2f\ \mathrm{J/m^2}$"%(temp,sA,sB,sC), fontsize=24)
fig.tight_layout(pad=1.0, w_pad=2.5, h_pad=2.5)
fig.subplots_adjust(top=0.95)

bar = FloatProgress(min=0, max=lattice**2)
display(bar)

for p in range(lattice):
    for q in range(lattice):
        rA = r0 * (p+1) # m
        rB = r0 * (q+1) # m
        rC = rB         # m
        
        allCr = []
        allNb = []
        allG = []
        allID = []
        points = []
        phases = []

        pool = Pool(12)
        for result in pool.imap(computeKernelLattice, range(density**2)):
            xcr, xnb, xni, fg, fd, fu, fh, fl = result
            f = (fg, fd, fu, fh, fl)

            # Accumulate (x, y, G) points for each phase
            if (fd**2 + fu**2 + fh**2 + fl**2) > epsilon:
                for n in range(len(f)):
                    allCr.append(rt3by2*xcr)
                    allNb.append(xnb+xcr/2)
                    allG.append(f[n])
                    allID.append(n)
        pool.close()
        pool.join()

        points = np.array([allCr, allNb, allG]).T

        hull = ConvexHull(points)

        X = [[],[],[],[], [], []]
        Y = [[],[],[],[], [], []]
        tielines = []

        for simplex in hull.simplices:
            for i in simplex:
                X[allID[i]].append(allNb[i])
                Y[allID[i]].append(allCr[i])
                for j in simplex:
                    if allID[i] != allID[j]:
                        tielines.append([[allNb[i], allNb[j]],
                                         [allCr[i], allCr[j]]])
        
        axs[p,q].set_xlim([0,1])
        axs[p,q].set_ylim([0,rt3by2])
        axs[p,q].set_xlabel(r'$r_\mu=%.1f\ \mathrm{nm}$'%(2.5*(q+1)), fontsize=18)
        axs[p,q].set_ylabel(r'$r_\delta=%.1f\ \mathrm{nm}$'%(2.5*(p+1)), fontsize=18)
        axs[p,q].plot(XS, YS, '-k', alpha=0.1)
        for tie in tielines:
            axs[p,q].plot(tie[0], tie[1], '-k', alpha=0.025)
        for i in range(len(labels)):
            axs[p,q].scatter(X[i], Y[i], color=colors[i], s=2.5)
        bar.value = (p*lattice) + q + 1

plt.show()

### Take derivatives of annoying functions

In [21]:
p = symbols('p', real=True)
h = Abs(p)**3 * (16*p**2 - 15*Abs(p) + 10)
hprime = simplify(diff(h, p))
#print h
print hprime


2*p*(8*p**3*sign(p) + 32*p**2*Abs(p) - 15*p**2 - 15*p*Abs(p)*sign(p) + 5*p*sign(p) + 10*Abs(p))
